# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [36]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [37]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [38]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [39]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = data_sframe[features]
    
    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe[output]

    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [40]:
def predict_output(feature_matrix, weights):
    ''' Returns an array of predictions
    
    inputs - 
        feature_matrix - 2-D array of dimensions data points by features
        coefficients - 1-D array of estimated feature coefficients
        
    output - 1-D array of predictions
    '''
    predictions = np.dot(feature_matrix, weights)
    print predictions
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [41]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [42]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [43]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [44]:
def normalize_features(feature_matrix):
    X = np.array(feature_matrix)
    norms = np.linalg.norm(X, axis=0)
    normalized_features = X / norms
    return normalized_features, norms

To test the function, run the following:

In [45]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [51]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [52]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [53]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [49]:
prediction = predict_output(simple_feature_matrix, weights)
print prediction.size

[ 0.02675867  0.04339256  0.01990703 ...,  0.02289873  0.03178473
  0.02289873]
21613


Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [50]:
ro = []
ro = simple_feature_matrix[:,0]*(output - prediction + weights[0]*simple_feature_matrix[:,0])
ro_0 = ro.sum()

for i in xrange(0,len(weights)):
    ro[i] = sum(simple_feature_matrix[:,i]*(output - prediction + weights[i]*simple_feature_matrix[:,i]))
    print ro


[  7.94003000e+07   3.65952501e+03   1.22437639e+03 ...,   2.73512771e+03
   2.72083645e+03   2.21067965e+03]
[  7.94003000e+07   8.79394708e+07   1.22437639e+03 ...,   2.73512771e+03
   2.72083645e+03   2.21067965e+03]
[  7.94003000e+07   8.79394708e+07   8.09666987e+07 ...,   2.73512771e+03
   2.72083645e+03   2.21067965e+03]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [57]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(feature_matrix[:,i]*(output - prediction + weights[i]*feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + (l1_penalty/2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - (l1_penalty/2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [17]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

[ 2.09696136  4.34943339]
0.425558846691


In [72]:
l1_penalty = 2.3e8
print lasso_coordinate_descent_step(2, simple_feature_matrix, output, weights, l1_penalty)

[ 0.02675867  0.04339256  0.01990703 ...,  0.02289873  0.03178473
  0.02289873]
0.0


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [18]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    notConverged = True
    while notConverged:
        notConverged = False
        weights = initial_weights
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            # use old_weights_i to compute change in coordinate
            delta_weights = np.absolute(old_weights_i - weights[i])
            if delta_weights > tolerance:
                notConverged = True
    
    return weights    

Using the following parameters, learn the weights on the sales dataset. 

In [73]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [74]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [75]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print 'Coordinate descent lasso weights: ', weights

[ 0.  0.  0. ...,  0.  0.  0.]
[ 540088.14190534  540088.14190534  540088.14190534 ...,  540088.14190534
  540088.14190534  540088.14190534]
[ 576467.92028475  619322.06600288  563827.48881394 ...,  571535.06897907
  589416.65496217  571535.06897907]
[ 574718.81592844  617572.96164656  562661.41924306 ...,  570368.99940819
  587667.55060585  570368.99940819]
[ 512560.15803997  555414.30375809  500502.7613546  ...,  508210.34151973
  525508.89271739  508210.34151973]
[ 543069.46554971  621862.54130049  520411.37727197 ...,  534582.79377391
  566877.44527297  534582.79377391]
[ 544818.56990603  623611.64565681  521577.44684285 ...,  535748.86334479
  568626.54962928  535748.86334479]
[ 489076.72601467  567869.80176546  465835.60295149 ...,  480007.01945343
  512884.70573793  480007.01945343]
[ 514542.32872802  623333.02123435  482452.98777291 ...,  502019.659087
  547414.33653569  502019.659087  ]
[ 514542.32872802  623333.02123435  482452.98777291 ...,  502019.659087
  547414.33653569  

In [76]:
prediction = predict_output(normalized_simple_feature_matrix, weights)
residual = output - prediction        
sq_residual = residual*residual
RSS = sq_residual.sum()
print RSS

[ 370053.87731138  632691.61911816  292585.19087916 ...,  339822.19480125
  449412.04390048  339822.19480125]
1.63049248148e+15


In [79]:
weights

array([ 21624998.36636353,  63157246.78545319,         0.        ])

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [23]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [24]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [26]:
(simple_feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
norm_simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [28]:
l1_penalty=1e7
tolerance=1
initial_weights = np.zeros(14)
weights1e7 = lasso_cyclical_coordinate_descent(norm_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print 'Coordinate descent lasso weights (1e7: L1): ', weights1e7

[ 0.  0.  0. ...,  0.  0.  0.]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 551978.94243611  567744.33520455  551978.94243611 ...,  570897.41375824
  570897.41375824  548825.86388242]
[ 572714.39308587  612905.4438231   565509.70260587 ...,  597783.20993971
  599013.27904606  566749.7280034 ]
[ 572714.39308587  612905.4438231   565509.70260587 ...,  597783.20993971
  599013.27904606  566749.7280034 ]
[ 572714.39308587  612905.4438231   565509.70260587 ...,  597783.20993971
  599013.27904606  566749.7280034 ]
[ 572714.39308587  612905.4438231   565509.70260587 ...,  597783.20993971
  599013.27904606  566749.7280034 ]
[ 572714.39308587  612905.4438231   565509.70260587 ...,  597783.20993971
  599013.27904606  566749.7280034 ]
[ 536470.08301471  576661.13375195  529265.39253471 ...,  561538.89986856
  562768.968974

In [81]:
weights1e7

array([ 24429600.60933346,         0.        ,         0.        ,
        48389174.35227959,         0.        ,         0.        ,
         3317511.16271979,   7329961.98489638,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [82]:
all_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [29]:
l1_penalty=1e8
tolerance=1
initial_weights = np.zeros(14)
weights1e8 = lasso_cyclical_coordinate_descent(norm_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print 'Coordinate descent lasso weights (1e8: L1): ', weights1e8

[ 0.  0.  0. ...,  0.  0.  0.]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.628221

***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [30]:
l1_penalty=1e4
tolerance=5e5
initial_weights = np.zeros(14)
weights1e4 = lasso_cyclical_coordinate_descent(norm_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print 'Coordinate descent lasso weights (1e4: L1): ', weights1e4

[ 0.  0.  0. ...,  0.  0.  0.]
[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
[ 565166.85155346  565166.85155346  556566.77710943 ...,  565166.85155346
  565166.85155346  556566.77710943]
[ 581887.19262656  602787.61896793  573287.11818252 ...,  606967.7042362
  606967.7042362   569107.03291425]
[ 602940.1850867   648640.32288536  587025.08783872 ...,  634265.22835825
  635514.13469063  587305.38232895]
[ 601939.98696613  647358.29902887  585254.82567841 ...,  634065.01170792
  635091.39608675  587114.9021205 ]
[ 558971.60098431  561421.52706523  542286.43969659 ...,  505159.85376247
  549154.62412312  501178.13015686]
[ 558971.60098431  561421.52706523  542286.43969659 ...,  505159.85376247
  549154.62412312  501178.13015686]
[ 558971.60098431  561421.52706523  542286.43969659 ...,  505159.85376247
  549154.62412312  501178.13015686]
[ 500961.52415772  503411.45023864  484276.36287001 ...,  447149.77693588
  491144.5472965

In [83]:
print weights1e4

[ 77779073.91265044 -22884012.25023264  15348487.08089934
  92166869.69883147  -2139328.08242773  -8818455.54409455
   6494209.73310656   7065162.05053201   4119079.21006769
  18436483.52618784 -14566678.54514407  -5528348.7517945  -83591746.20730424
   2784276.46012856]


In [84]:
all_features

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [31]:
weights1e7_norm = weights1e7 / norms
weights1e8_norm = weights1e8 / norms
weights1e4_norm = weights1e4 /norms
print weights1e7_norm[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [32]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [33]:
prediction = predict_output(test_feature_matrix, weights1e7_norm)
residual = test_output - prediction
sq_residual = residual*residual
RSS = sq_residual.sum()
print '1e7 RSS: ', RSS

[ 415969.49505194  868753.14566658  461138.38280148 ...,  591805.52236267
  557928.85655051  349829.3379901 ]
1e7 RSS:  2.75962079909e+14


In [34]:
prediction = predict_output(test_feature_matrix, weights1e8_norm)
residual = test_output - prediction
sq_residual = residual*residual
RSS = sq_residual.sum()
print '1e8 RSS: ', RSS

[ 539366.62822135  539366.62822135  539366.62822135 ...,  539366.62822135
  539366.62822135  539366.62822135]
1e8 RSS:  5.37166150034e+14


In [35]:
prediction = predict_output(test_feature_matrix, weights1e4_norm)
residual = test_output - prediction
sq_residual = residual*residual
RSS = sq_residual.sum()
print '1e4 RSS: ', RSS

[ 343705.8378415   929838.74455382  457769.54658623 ...,  606337.25950945
  535610.44896873  205789.18826781]
1e4 RSS:  2.2778100476e+14


***QUIZ QUESTION***

Which model performed best on the test data?